Setting up PySpark in Colab

Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to download Java

In [ ]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Next, we will install Apache Spark 3.0.1 with Hadoop 2.7

In [2]:
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz


Now, we just need to unzip that folder.

In [3]:
!tar xf spark-3.0.3-bin-hadoop3.2.tgz

There is one last thing that we need to install and that is the findspark library. It will locate Spark on the system and import it as a regular library.

In [ ]:
!pip install -q findspark
!pip install pyspark==3.0.2

Now that we have installed all the necessary dependencies in Colab, it is time to set the environment path. This will enable us to run Pyspark in the Colab environment.

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] =  "/content/spark-3.0.3-bin-hadoop3.2"

We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.

In [6]:
import findspark
findspark.init()

If you want to know the location where Spark is installed, use findspark.find()

In [7]:
findspark.find()

'/content/spark-3.0.3-bin-hadoop3.2'

Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.

You can give a name to the session using appName() and add some configurations with config() if you wish.

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Finally, print the SparkSession variable.

In [ ]:
spark

In [ ]:
#we need to load the dataset. We will use the read.csv module. 
#The inferSchema parameter provided will enable Spark to automatically determine the data type for each column but it has to go over the data once.
# If you don’t want that to happen, then you can instead provide the schema explicitly in the schema parameter.

df = spark.read.csv("/content/data.csv", header=True, inferSchema= True)
df.printSchema()

In [ ]:
df.show(5, truncate=False)

In [ ]:
#If you didn't set inderShema to True, here is what is happening to the type. There are all in string.
df_string = spark.read.csv("/content/data.csv", header=True, inferSchema=  False)
df_string.printSchema()

In [ ]:
#You can select and show the rows with select and the names of the features. Below, gender and churn are selected.
df.select('gender','churn').show(5)

In [ ]:
#To get a summary statistics, of the data, you can use describe(). It will compute the :count, mean, standarddeviation, min, max
df.describe().show()

In [ ]:
#Distinct values for Categorical columns
#The distinct() will come in handy when you want to determine the unique values in the categorical columns in the dataframe.

df.select("PaymentMethod").distinct().show()

Map reduce example:


In [ ]:
# For map reduce we need to create SparkContext so that we can read a textfile and perform mapping function on it. 
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
text_file = sc.textFile("/content/icp4.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
for x in counts.collect():
    print (x)

Actions

In [ ]:
df.count()

 Finding First Row

In [ ]:
df.first()

Finding Required rows using take function


In [ ]:
df.take(5)

In [ ]:
df.describe().toPandas().transpose()

In [ ]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns)

In [ ]:
df.select('InternetService').take(5)#action on next block 

In [ ]:
df.select('InternetService').first()

In [ ]:
#Collect:This task helps us present the PaymentMethod column's elements as an array to the drivernode

df.select('PaymentMethod').collect()

In [ ]:
#Counting Null records in the partner column

df.filter(df["partner"].isNull()).count()

In [ ]:
#Min: using an aggregate function min, i will display the minimum monthly charge on the df

df.agg({"MonthlyCharges":"min","MonthlyCharges":"min"}).show()

In [ ]:
#get last 3 element using tail function

action = df.tail(3)
print("action: Getting last 3 data points from data set")
action

In [ ]:
#Printing summary

act = df.summary()
print("action: Getting all the summary of data set")
act.show()

**Tranformations**

Replacing the data


In [ ]:
df.replace('Male',None).show()

In [ ]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("nullValue", "NA").option("timestampFormat", "yyyy-MM-dd'T'HH:mm?:ss").option("mode", "failfast").option(
    "path",
    "/content/data.csv"
  ).load()
df.show()

Finding required data using filter


In [ ]:

df.filter(df.gender == 'Male').collect()

Grouping the data and showing count.


In [ ]:

op=df.groupBy('gender').count()
op.show()


A lambda function is a small anonymous function.A lambda function can take any number of arguments, but can only have one expression.The power of lambda is better shown when you use them as an anonymous function inside another function.

The truncate() method resizes the file to the given number of bytes.

In [ ]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import UserDefinedFunction

binary_map = {'Yes':1.0, 'No':0.0, 'True':1.0, 'False':0.0}
toNum = UserDefinedFunction(lambda k: binary_map[k], DoubleType())

df = df.drop('customerID').withColumn('Churn', toNum(df['Churn'])).withColumn('Partner', toNum(df['Partner'])).withColumn('Dependents', toNum(df['Dependents'])).cache()

df.show()

In [ ]:
file = sc.textFile("/content/data.csv").map(lambda line : line.split(","))
#Filter function : Return a new dataset formed by selecting those elements of the source on which func returns true.
#ex: customer ID ; 7795-CFOCW
data1 = file.filter(lambda line: "One year" in line).count()
print(data1)

In [ ]:
# MAP function = applies transformation function on dataset and returns same number of elements in distributed dataset.

fm = file.map(lambda column : (column [20], 1)).reduceByKey(lambda x, y: x + y)
fm.collect()

In [ ]:
#Here i am using the filter function to show only information for those who pay with electronic check and have one year contracts

df.filter("PaymentMethod='Electronic check' and contract='One year'").show()

In [ ]:

#Groupby: telling spark to group my data by customer ID and internetservice and i am getting the maximum charge for each internetservice and customer that pays for it

max_date=df.agg({"TotalCharges":"max"})
max_date.show()

In [ ]:
 #Map and FlatMapflatmap splits each record by space in an RDD and finally flattens it. Resulting RDD consists of a single word on each record.The map function iterates over every line in RDD and split into new RDD. Using map() transformation we take in any function, and that function is applied to every element of RDD.

df.rdd.map(lambda line: line.StreamingTV  .split(" ")).take(50)

In [ ]:
df.rdd.flatMap(lambda line: line.StreamingTV  .split(" ")).take(50)

In [ ]:
df.printSchema()

In [ ]:
#Startswith: Using Startswith to show records that start with F from the internetservice column

from pyspark.sql.functions import col
new_df=df.filter(col("InternetService").startswith("F")).show()

In [ ]:
#groupby: Here, we are doing a groupBy on the Paymentmethod column and then finding the sum of Tenure for each paymentMethod using sum function

df.groupBy("PaymentMethod").sum("Tenure").show(truncate=False)